In [22]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import plotly.express as px
from logger import logger
has_mps = torch.backends.mps.is_built()
# device = "cpu"
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"++++++  Using device: {device}   ++++++")
logger.info("without init weights")
logger.info("train on 36, 37, 38, test on 35")

2025-03-09 20:00:25,317 - logger - INFO - ++++++  Using device: cuda   ++++++
INFO:logger:++++++  Using device: cuda   ++++++
2025-03-09 20:00:25,317 - logger - INFO - without init weights
INFO:logger:without init weights
2025-03-09 20:00:25,317 - logger - INFO - train on 36, 37, 38, test on 35
INFO:logger:train on 36, 37, 38, test on 35


In [23]:
fn = "./datasets/CALCE/CALCE.csv"
df = pd.read_csv(fn)


spots_train = df["capacity_CS2_36"].to_numpy().reshape(-1, 1)
spots_train.shape


(882, 1)

In [24]:
scaler = StandardScaler()
spots_train = scaler.fit_transform(spots_train).flatten().tolist()
leng, min_val, max_val = len(spots_train), min(spots_train), max(spots_train)
spots_train = np.linspace(min_val, max_val, leng).tolist()

In [26]:
# Sequence Data Preparation
SEQUENCE_SIZE = 32

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(len(obs) - seq_size):
        window = obs[i:(i + seq_size)]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)

x_train, y_train = to_sequences(SEQUENCE_SIZE, spots_train)


In [27]:
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [28]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, 1)
    #     self.apply(self._init_weights) # what's the weights if not initialized?
    #     # see link https://chatgpt.com/c/67ce20e9-7774-8009-85d9-dcce0feca042
        
    # def _init_weights(self, module):
    #     if isinstance(module, nn.Linear):
    #         torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    #         if module.bias is not None:
    #             torch.nn.init.zeros_(module.bias)
    #     elif isinstance(module, nn.Embedding):
    #         torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

model = TransformerModel().to(device)


c:\Users\micha\miniconda3\envs\py310\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [29]:
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 1000
early_stop_count = 0
min_val_loss = float('inf')


c:\Users\micha\miniconda3\envs\py310\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [30]:
for epoch in range(epochs):
    model.train()
    train_losses = []
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())
        loss.backward() # calculate gradients
        optimizer.step() # update weights based on gradients and learning rate

    train_loss = np.mean(train_losses)

    scheduler.step(train_loss)

    if train_loss < min_val_loss:
        min_val_loss = train_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 5:
        logger.info("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}")

Epoch 1/1000, Train Loss: 0.2915
Epoch 2/1000, Train Loss: 0.0480
Epoch 3/1000, Train Loss: 0.0315
Epoch 4/1000, Train Loss: 0.0305
Epoch 5/1000, Train Loss: 0.0241
Epoch 6/1000, Train Loss: 0.0219
Epoch 7/1000, Train Loss: 0.0247
Epoch 8/1000, Train Loss: 0.0210
Epoch 9/1000, Train Loss: 0.0179
Epoch 10/1000, Train Loss: 0.0180
Epoch 11/1000, Train Loss: 0.0179
Epoch 12/1000, Train Loss: 0.0223
Epoch 13/1000, Train Loss: 0.0163
Epoch 14/1000, Train Loss: 0.0171
Epoch 15/1000, Train Loss: 0.0153
Epoch 16/1000, Train Loss: 0.0165
Epoch 17/1000, Train Loss: 0.0155
Epoch 18/1000, Train Loss: 0.0160
Epoch 19/1000, Train Loss: 0.0149
Epoch 20/1000, Train Loss: 0.0174
Epoch 21/1000, Train Loss: 0.0127
Epoch 22/1000, Train Loss: 0.0132
Epoch 23/1000, Train Loss: 0.0113
Epoch 24/1000, Train Loss: 0.0155
Epoch 25/1000, Train Loss: 0.0139
Epoch 26/1000, Train Loss: 0.0122
Epoch 27/1000, Train Loss: 0.0108
Epoch 28/1000, Train Loss: 0.0114
Epoch 29/1000, Train Loss: 0.0114
Epoch 30/1000, Train Lo

2025-03-09 20:03:03,288 - logger - INFO - Early stopping!
INFO:logger:Early stopping!


Epoch 31/1000, Train Loss: 0.0150


In [31]:
# Evaluation
from sklearn.metrics import mean_absolute_error
# Evaluation
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.squeeze().tolist())
        actuals.extend(y_batch.squeeze().tolist())
        
# Convert to numpy arrays and inverse transform
predictions = np.array(predictions).reshape(-1, 1)
actuals = np.array(actuals).reshape(-1, 1)

predictions_inv = scaler.inverse_transform(predictions)
actuals_inv = scaler.inverse_transform(actuals)

# Compute Scores
rmse = np.sqrt(np.mean((predictions_inv - actuals_inv) ** 2))
mae = mean_absolute_error(actuals_inv, predictions_inv)
re = np.mean(np.abs((actuals_inv - predictions_inv) / actuals_inv))  # Percentage form

logger.info(f"SEQUENCE_SIZE: {SEQUENCE_SIZE}")

logger.info(f"Score (Relative Error): {re:.4f}")
logger.info(f"Score (MAE): {mae:.4f}")
logger.info(f"Score (RMSE): {rmse:.4f}")

2025-03-09 20:03:33,087 - logger - INFO - SEQUENCE_SIZE: 32
INFO:logger:SEQUENCE_SIZE: 32
2025-03-09 20:03:33,088 - logger - INFO - Score (Relative Error): 0.0215
INFO:logger:Score (Relative Error): 0.0215
2025-03-09 20:03:33,088 - logger - INFO - Score (MAE): 0.0138
INFO:logger:Score (MAE): 0.0138
2025-03-09 20:03:33,089 - logger - INFO - Score (RMSE): 0.0158
INFO:logger:Score (RMSE): 0.0158
